In [206]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt

In [207]:
def author_url(author):
    return f"https://github.com/{author}"

In [208]:
bot_names = ["stale", "codecov", "jupyterlab-dev-mode", "henchbot"]
comments = pd.read_csv('./data/comments.csv', index_col=0).query('author not in @bot_names')
issues = pd.read_csv('./data/issues.csv', index_col=0).query('author not in @bot_names')
prs = pd.read_csv('./data/prs.csv', index_col=0).query('author not in @bot_names')

In [209]:
start_date = comments['updatedAt'].min()
end_date = comments['updatedAt'].max()

renderer = "kaggle"
alt.renderers.enable(renderer)

RendererRegistry.enable('kaggle')

In [210]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [211]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [212]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/nteract](https://github.com/nteract)

In [213]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-06-21T00:44:53Z** to **2019-07-20T22:53:25Z**

In [214]:
merged = prs.query('state == "MERGED"')

## Commentors

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [215]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                           repo  author
  58          nteract                        nteract      15
  57          nteract                       hydrogen      13
  55          nteract                      bookstore       9
  60          nteract                      papermill       9
  59          nteract                     nteract.io       5
  56          nteract                       commuter       3
  63          nteract                  vega-embed-v3       2
  61          nteract                      scrapbook       1
  62          nteract                           vdom       1
  45       jupyterlab                     jupyterlab      52
  47       jupyterlab       jupyterlab-data-explorer       7
  48       jupyterlab                jupyterlab-demo       5
  49       jupyterlab                 jupyterlab-git       5
  44       jupyterlab              jupyter-renderers       2
  46       jupyterlab          jupyterlab-commenting       2
  53       jupyterlab  scipy2019-jupyterlab-tutorial       2
  41       jupyterlab                       debugger       1
  42       jupyterlab      extension-cookiecutter-js       1
  43       jupyterlab      extension-cookiecutter-ts       1
  50       jupyterlab              jupyterlab-github       1
  51       jupyterlab              jupyterlab_server       1
  52       jupyterlab     mimerender-cookiecutter-ts       1
  54       jupyterlab             theme-cookiecutter       1
  33       jupyterhub            mybinder.org-deploy      21
  29       jupyterhub                     jupyterhub      11
  37       jupyterhub        the-littlest-jupyterhub       8
  31       jupyterhub          kerberosauthenticator       4
  36       jupyterhub                   team-compass       4
  38       jupyterhub                  traefik-proxy       3
  27       jupyterhub                      binderhub       2
  ..              ...                            ...     ...
  30       jupyterhub                 jupyterlab-hub       1
  32       jupyterhub               ltiauthenticator       1
  34       jupyterhub                 oauthenticator       1
  35       jupyterhub                    sudospawner       1
  20  jupyter-widgets                     ipyleaflet      16
  21  jupyter-widgets                     ipywidgets      16
  25  jupyter-widgets                       tutorial      13
  22  jupyter-widgets             jupyterlab-sidecar       1
  23  jupyter-widgets                   midicontrols       1
  24  jupyter-widgets                     traittypes       1
  16          jupyter                       notebook      21
  18          jupyter                    repo2docker      15
  19          jupyter                      telemetry      11
  5           jupyter             enterprise_gateway       9
  10          jupyter              jupyter.github.io       7
  11          jupyter                 jupyter_client       7
  4           jupyter                  docker-stacks       5
  8           jupyter                   jupyter-book       4
  9           jupyter                 jupyter-sphinx       4
  12          jupyter                 jupyter_server       4
  14          jupyter                      nbconvert       3
  15          jupyter                       nbviewer       3
  17          jupyter                      qtconsole       3
  6           jupyter                     governance       1
  7           jupyter                        jupyter       1
  13          jupyter                          nb2kg       1
  2           ipython                        ipython       8
  3           ipython                      traitlets       2
  1           ipython                      ipykernel       1
  0   binder-examples                     data-quilt       1
  
  [64 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['nteract', 'hydrogen', 'bookstor

### A list of merged PRs by project

In [216]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

Authoring and merging stats by repo

In [217]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [218]:
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                org        repo     username  authored authorAssociation
    0  binder-examples  data-quilt  matheusmota         1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for binder-examples in the last 29 days',
    width: 1000
  }), Chart({
    data:        org       repo        username  authored authorAssociation
    1  ipython  ipykernel        steff456         1              NONE
    2  ipython    ipython         Carreau         4            MEMBER
    3  ipython    ipython      OriolAbril         1       CONTRIBUTOR
    4  ipython    ipython  brandonwillard         1       CONTRIBUTOR
    5  ipython    ipython         sunpoet         1       CONTRIBUTOR
    6  ipython    ipython      sysadmin75         1              NONE
    7  ipython  traitlets           Jamim         2       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for ipython in the last 29 days',
    width: 1000
  }), Chart({
    data:         org                repo        username  authored authorAssociation
    8   jupyter       docker-stacks    danielballan         1       CONTRIBUTOR
    9   jupyter       docker-stacks     jamesdbrock         3       CONTRIBUTOR
    10  jupyter       docker-stacks           minrk         1            MEMBER
    11  jupyter  enterprise_gateway        IMAM9AIS         1       CONTRIBUTOR
    12  jupyter  enterprise_gateway           clkao         1       CONTRIBUTOR
    13  jupyter  enterprise_gateway     kevin-bates         4            MEMBER
    14  jupyter  enterprise_gateway        lresende         1            MEMBER
    15  jupyter  enterprise_gateway   vikasgarg1996         2       CONTRIBUTOR
    16  jupyter          governance       ellisonbg         1            MEMBER
    17  jupyter             jupyter       choldgraf         1            MEMBER
    18  jupyter        jupyter-book           cczhu         1       CONTRIBUTOR
    19  jupyter        jupyter-book       choldgraf         1            MEMBER
    20  jupyter        jupyter-book         emdupre         1      COLLABORATOR
    21  jupyter        jupyter-book            habi         1       CONTRIBUTOR
    22  jupyter      jupyter-sphinx        akhmerov         3            MEMBER
    23  jupyter      jupyter-sphinx        jbweston         1            MEMBER
    24  jupyter   jupyter.github.io          afshin         2            MEMBER
    25  jupyter   jupyter.github.io    davidanthoff         1       CONTRIBUTOR
    26  jupyter   jupyter.github.io         jMuzsik         1       CONTRIBUTOR
    27  jupyter   jupyter.github.io      jasongrout         1            MEMBER
 

In [219]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                org        repo username  closed
    0  binder-examples  data-quilt  betatim       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for binder-examples in the last 29 days',
    width: 1000
  }), Chart({
    data:        org       repo  username  closed
    1  ipython  ipykernel     minrk       1
    2  ipython    ipython   Carreau       8
    3  ipython  traitlets  rmorshea       2,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 29 days',
    width: 1000
  }), Chart({
    data:         org                repo       username  closed
    4   jupyter       docker-stacks          minrk       5
    5   jupyter  enterprise_gateway    kevin-bates       7
    6   jupyter  enterprise_gateway       lresende       2
    7   jupyter          governance      choldgraf       1
    8   jupyter             jupyter      choldgraf       1
    9   jupyter        jupyter-book      choldgraf       4
    10  jupyter      jupyter-sphinx       akhmerov       3
    11  jupyter      jupyter-sphinx       jbweston       1
    12  jupyter   jupyter.github.io        Carreau       1
    13  jupyter   jupyter.github.io         afshin       1
    14  jupyter   jupyter.github.io      choldgraf       2
    15  jupyter   jupyter.github.io      ellisonbg       1
    16  jupyter   jupyter.github.io     jasongrout       2
    17  jupyter      jupyter_client          MSeal       1
    18  jupyter      jupyter_client  SylvainCorlay       4
    19  jupyter      jupyter_client          minrk       1
    20  jupyter      jupyter_client         rgbkrk       1
    21  jupyter      jupyter_server  SylvainCorlay       2
    22  jupyter      jupyter_server       rolweber       2
    23  jupyter               nb2kg    kevin-bates       1
    24  jupyter           nbconvert          MSeal       3
    25  jupyter            nbviewer      choldgraf       1
    26  jupyter            nbviewer        parente       2
    27  jupyter            notebook  SylvainCorlay       1
    28  jupyter            notebook         ivanov       1
    29  jupyter            notebook       lresende       4
    30  jupyter            notebook          minrk       2
    31  jupyter            notebook      takluyver      13
    32  jupyter           qtconsole     ccordoba12       3
    33  jupyter         repo2docker        betatim      10
    34  jupyter         repo2docker          minrk       4
    35  jupyter         repo2docker      yuvipanda       1
    36  jupyter           telemetry        Zsailer       6
    37  jupyter           telemetry      yuvipanda       5,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter in the last 29 days',
    width: 1000
  }), Chart({
    data:                 org                repo         username  closed
    38  jupyter-widgets          ipyleaflet    SylvainCorlay       3
    39  jupyter-widgets          ipyleaflet      martinRenou      13
    40  jupyter-widgets          ipywidgets    SylvainCorlay      14
    41  jupyter-widgets          ipywidgets       jasongrout       1
    42  jupyter-widgets          ipywidgets          vidartf       1
    43  jupyter-widgets  jupyterlab-sidecar    SylvainCorlay       1
    44  jupyter-widgets        midicontrols       jasongrout       1
    45  jupyter-widgets          traittypes    SylvainCorlay       1
    46  jupyter-widgets            tutorial    SylvainCorlay       1
    47  jupyter-widgets  

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [220]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [221]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                         repo  number  \
  0           ipython                    ipykernel       1   
  1           ipython                  ipyparallel       1   
  2           ipython                      ipython       7   
  3           ipython             ipython-in-depth       1   
  4           jupyter                accessibility       1   
  5           jupyter                docker-stacks       6   
  6           jupyter           enterprise_gateway       3   
  7           jupyter                      jupyter       4   
  8           jupyter                 jupyter-book       9   
  9           jupyter                jupyter-drive       1   
  10          jupyter               jupyter-sphinx       6   
  11          jupyter               jupyter_client       1   
  12          jupyter               jupyter_server       2   
  13          jupyter                        nb2kg       2   
  14          jupyter                    nbconvert       4   
  15          jupyter                       nbdime       3   
  16          jupyter                     nbgrader       4   
  17          jupyter                     nbviewer       1   
  18          jupyter                     notebook      27   
  19          jupyter                    qtconsole       2   
  20          jupyter                  repo2docker      12   
  21          jupyter                     testpath       1   
  22  jupyter-widgets                   ipyleaflet       4   
  23  jupyter-widgets                   ipywidgets      27   
  24  jupyter-widgets           jupyterlab-sidecar       3   
  25  jupyter-widgets                    pythreejs       2   
  26       jupyterhub                 batchspawner       3   
  27       jupyterhub                    binderhub      10   
  28       jupyterhub                dockerspawner       4   
  29       jupyterhub                   helm-chart       1   
  ..              ...                          ...     ...   
  32       jupyterhub                   jupyterhub      17   
  33       jupyterhub        kerberosauthenticator       1   
  34       jupyterhub                  kubespawner       4   
  35       jupyterhub            ldapauthenticator       4   
  36       jupyterhub          mybinder.org-deploy       4   
  37       jupyterhub          nativeauthenticator       1   
  38       jupyterhub               systemdspawner       3   
  39       jupyterhub                 team-compass       7   
  40       jupyterhub      the-littlest-jupyterhub      11   
  41       jupyterhub                traefik-proxy       2   
  42       jupyterhub                  wrapspawner       3   
  43       jupyterhub       zero-to-jupyterhub-k8s      11   
  44       jupyterlab    extension-cookiecutter-ts       1   
  45       jupyterlab            jupyter-renderers       2   
  46       jupyterlab                   jupyterlab      74   
  47       jupyterlab          jupyterlab-celltags       3   
  48       jupyterlab        jupyterlab-commenting      12   
  49       jupyterlab     jupyterlab-data-explorer      11   
  50       jupyterlab               jupyterlab-git       7   
  51       jupyterlab  jupyterlab-metadata-service       1   
  52       jupyterlab               jupyterlab-toc       1   
  53       jupyterlab              jupyterlab_apod       1   
  54       jupyterlab               jupyterlab_app       1   
  55       jupyterlab                 team-compass       3   
  56          nteract                    bookstore       3   
  57          nteract                  coffee_boat       1   
  58          nteract                     hydrogen      11   
  59          nteract                      nteract       4   
  60          nteract                    papermill       7   
  61          nteract                     semiotic       4   
  
                                    org/repo  
  0                        ipython/ipykernel  
  1                      ipython/ipyparallel 

In [222]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                       repo  number  \
  0   binder-examples                 demo-julia       1   
  1           ipython                  ipykernel       1   
  2           ipython                    ipython      16   
  3           ipython                  traitlets       2   
  4           jupyter              docker-stacks       2   
  5           jupyter         enterprise_gateway       6   
  6           jupyter                    jupyter       1   
  7           jupyter               jupyter-book       6   
  8           jupyter             jupyter-sphinx      10   
  9           jupyter          jupyter.github.io       1   
  10          jupyter             jupyter_client       4   
  11          jupyter             jupyter_server       1   
  12          jupyter                      nb2kg       1   
  13          jupyter                  nbconvert       2   
  14          jupyter                     nbdime       1   
  15          jupyter                   nbgrader       1   
  16          jupyter                   nbviewer       3   
  17          jupyter                   notebook      46   
  18          jupyter                  qtconsole       2   
  19          jupyter                repo2docker       7   
  20          jupyter                  telemetry       3   
  21  jupyter-widgets                 ipyleaflet      10   
  22  jupyter-widgets                 ipywidgets      21   
  23  jupyter-widgets         jupyterlab-sidecar       1   
  24  jupyter-widgets               midicontrols       1   
  25  jupyter-widgets                  pythreejs       1   
  26  jupyter-widgets                   tutorial       1   
  27       jupyterhub               batchspawner       1   
  28       jupyterhub                  binderhub       5   
  29       jupyterhub              dockerspawner       3   
  30       jupyterhub       jupyter-server-proxy       1   
  31       jupyterhub                 jupyterhub       6   
  32       jupyterhub   jupyterhub-deploy-docker       1   
  33       jupyterhub      kerberosauthenticator       1   
  34       jupyterhub                kubespawner       1   
  35       jupyterhub        mybinder.org-deploy       5   
  36       jupyterhub             oauthenticator       1   
  37       jupyterhub               team-compass      16   
  38       jupyterhub    the-littlest-jupyterhub       5   
  39       jupyterhub              traefik-proxy       4   
  40       jupyterhub     zero-to-jupyterhub-k8s      12   
  41       jupyterlab  extension-cookiecutter-js       1   
  42       jupyterlab          jupyter-renderers       7   
  43       jupyterlab                 jupyterlab      68   
  44       jupyterlab   jupyterlab-data-explorer       2   
  45       jupyterlab             jupyterlab-git       4   
  46       jupyterlab             jupyterlab-toc       2   
  47          nteract                  bookstore       5   
  48          nteract                   commuter       1   
  49          nteract                   hydrogen      35   
  50          nteract            meeting-minutes       1   
  51          nteract                    nteract      25   
  52          nteract                  papermill       6   
  53          nteract                  scrapbook       1   
  54          nteract                   semiotic       4   
  55          nteract                       vdom       1   
  
                                  org/repo  
  0             binder-examples/demo-julia  
  1                      ipython/ipykernel  
  2                        ipython/ipython  
  3                      ipython/traitlets  
  4                  jupyter/docker-stacks  
  5             jupyter/enterprise_gateway  
  6                        jupyter/jupyter  
  7                   jupyter/jupyter-book  
  8                 jupyter/jupyter-sphinx  
  9              jupyter/jupyter.github.io  
  10                jupyter/jupyter_client  
  11                jup

### A list of recent issues

In [223]:
# Add comment count data to issues and PRs
comment_counts = comments.groupby(['org', 'repo', 'issue_id']).count().iloc[:, 0].to_frame()
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [224]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for (org, repo), i_issues in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        
        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)
    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 29 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [225]:
commentors = (
    comments
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [226]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                org        repo        author authorAssociation  count
    2  binder-examples  demo-julia     choldgraf            MEMBER      2
    5  binder-examples  jupyterlab   cornhundred              NONE      2
    0  binder-examples  data-quilt       betatim            MEMBER      1
    1  binder-examples  data-quilt   matheusmota              NONE      1
    3  binder-examples  demo-julia  davidanthoff       CONTRIBUTOR      1
    4  binder-examples  jupyterlab     choldgraf            MEMBER      1,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 29 days',
    width: 1000
  }), Chart({
    data:         org         repo                 author authorAssociation  count
    12  ipython      ipython                Carreau            MEMBER     48
    40  ipython      ipython               gpotter2       CONTRIBUTOR      6
    14  ipython      ipython         LucianaMarques       CONTRIBUTOR      4
    28  ipython      ipython             ccordoba12       CONTRIBUTOR      4
    78  ipython    traitlets                  minrk            MEMBER      3
    79  ipython    traitlets               rmorshea      COLLABORATOR      3
    6   ipython    ipykernel             ccordoba12       CONTRIBUTOR      2
    36  ipython      ipython  faultdiagnosistoolbox              NONE      2
    37  ipython      ipython               fehiepsi              NONE      2
    42  ipython      ipython             jamesmyatt              NONE      2
    47  ipython      ipython               jseabold       CONTRIBUTOR      2
    62  ipython      ipython              tacaswell       CONTRIBUTOR      2
    64  ipython      ipython             thrasibule              NONE      2
    66  ipython      ipython               voronind              NONE      2
    67  ipython      ipython                wenjuno              NONE      2
    70  ipython      ipython              ztaylor96              NONE      2
    75  ipython    traitlets                  Jamim       CONTRIBUTOR      2
    7   ipython    ipykernel                hroncok       CONTRIBUTOR      1
    8   ipython    ipykernel                  kribu              NONE      1
    9   ipython    ipykernel                  minrk            MEMBER      1
    10  ipython  ipyparallel         stuartthomas25              NONE      1
    11  ipython      ipython           Benjamin-Lee              NONE      1
    13  ipython      ipython            ElieGouzien              NONE      1
    15  ipython      ipython             OriolAbril       CONTRIBUTOR      1
    16  ipython      ipython                   Qix-              NONE      1
    17  ipython      ipython               Scowley4              NONE      1
    18  ipython      ipython                   Suor              NONE      1
    19  ipython      ipython               YahavFes              NONE      1
    20  ipython      ipython               aakhmetz              NONE      1
    21  ipython      ipython           alessioarena              NONE      1
    22  ipython      ipython             alexmojaki              NONE      1
    23  ipython      ipython             anentropic              NONE      1
    24  ipython      ipython                anntzer       CONTRIBUTOR      1
    25  ipython      ipython      

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [227]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comments.append(i_comments.loc[ix_min])
first_comments = pd.concat(first_comments, axis=1).T

In [228]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)

In [229]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                org       author authorAssociation  createdAt  updatedAt  url  \
    0  binder-examples      betatim            MEMBER          1          1    1   
    1  binder-examples    choldgraf            MEMBER          1          1    1   
    2  binder-examples  cornhundred              NONE          1          1    1   
    
       repo  issue_id  n_first_responses  
    0     1         1                  1  
    1     1         1                  1  
    2     1         1                  1  ,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 29 days',
    width: 1000
  }), Chart({
    data:         org                 author authorAssociation  createdAt  updatedAt  \
    4   ipython                Carreau            MEMBER         20         20   
    36  ipython                  minrk            MEMBER          2          2   
    3   ipython           Benjamin-Lee              NONE          1          1   
    5   ipython                  Jamim       CONTRIBUTOR          1          1   
    6   ipython             OriolAbril       CONTRIBUTOR          1          1   
    7   ipython               Scowley4              NONE          1          1   
    8   ipython                   Suor              NONE          1          1   
    9   ipython               YahavFes              NONE          1          1   
    10  ipython             alexmojaki              NONE          1          1   
    11  ipython             anentropic              NONE          1          1   
    12  ipython             ccordoba12       CONTRIBUTOR          1          1   
    13  ipython            craymichael              NONE          1          1   
    14  ipython           davidmakovoz              NONE          1          1   
    15  ipython             denfromufa              NONE          1          1   
    16  ipython                dickmao              NONE          1          1   
    17  ipython         farisachugthai              NONE          1          1   
    18  ipython  faultdiagnosistoolbox              NONE          1          1   
    19  ipython              gdhameeja              NONE          1          1   
    20  ipython            germanesosa              NONE          1          1   
    21  ipython               gpotter2       CONTRIBUTOR          1          1   
    22  ipython                hroncok       CONTRIBUTOR          1          1   
    23  ipython             jamesmyatt              NONE          1          1   
    24  ipython               janusson              NONE          1          1   
    25  ipython             jasongrout            MEMBER          1          1   
    26  ipython                 jdatin              NONE          1          1   
    27  ipython              jmansilla              NONE          1          1   
    28  ipython         joaomarcosgris              NONE          1          1   
    29  ipython              keflavich              NONE          1          1   
    30  ipython                  kernc              NONE          1          1   
    31  ipython                  kribu              NONE          1          1   
    32  ipython                leogout              NONE          1          1   
    33  ipython

## List of all contributors per organization

In [230]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments['author']
    authors_prs = prs['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str))
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are all participants across grou and pull requests in each org in the last {n_days} days"))
display(tabs)

Here are all participants across grou and pull requests in each org in the last 29 days

In [231]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>